In [13]:
import os, glob
import pandas as pd
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import POSStopFilter, LowerCaseFilter

In [14]:
wakati = True
target_dir = 'scripts'
csv_out = 'scripts.csv'

In [15]:
token_filters = [ POSStopFilter(['助詞','助動詞']),
                  LowerCaseFilter(),
                ]
tokenizer = Tokenizer()
analyzer = Analyzer(tokenizer=tokenizer, token_filters=token_filters)

In [16]:
file_pat = '*.txt'
dir_pat = os.path.join(target_dir, '*')

text = []
for d in glob.glob(dir_pat):  # for-loop of directories in target_dir
    if not os.path.isdir(d): continue  # skip if d is not a directory
    category = os.path.basename(d)  # use directory name as the category name
    full_pat = os.path.join(d, file_pat)  # generate full path name for text files
    n_files = 0  # counter for number of files
    for file in glob.glob(full_pat):  # for-loop of text files
        n_files += 1
        with open(file, 'r', encoding='utf-8') as f:
            lines = f.read().splitlines()  # if you want to omit newline character
            body = ' '.join(lines[2:])  # 1st line: URL, 2nd line: Date, so skip these lines
            if wakati:  # add space between words (wakati-gaki)
                body = ' '.join([t.base_form for t in analyzer.analyze(body)])
            text.append([category, body])
    print(n_files, 'files read for', category)

5 files read for manzai
8 files read for rakugo


In [17]:
df = pd.DataFrame(text, columns=['category', 'text'])
print(df.shape)
print(df.info())
display(df.head())
display(df.tail())

(13, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  13 non-null     object
 1   text      13 non-null     object
dtypes: object(2)
memory usage: 336.0+ bytes
None


,category,text
0,manzai,こ ける お願い しま ー する ありがとう ー えー 先日 僕 たち 、 寝る...
1,manzai,えー 、 伊藤 畠中 いう 。 是非 、 個人 名 方 覚える 帰る くださる 。 あの...
2,manzai,畠中 : あのー こないだ 本当 参る ちゃう ん 伊藤 : うん 畠中...
3,manzai,きゃあ ああ 、 嫌 喉 きゃあ お前 よろしく お願い する よろしく おねがい ...
4,manzai,ありがとう 〜 ！ 駒場 ：( 客席 何 受け取る 振り ) 内海 ： あぁ 〜 あ...


,category,text
8,rakugo,八 五 郎 \t いる 、 じゃ 、 おじゃま する 。 、 ただ 酒 ？ ご 隠居 \...
9,rakugo,番頭 \t 悪い ん 、 ちょい 手紙 、 届ける くる 欲しい ん 定吉 \t 手紙 ...
10,rakugo,● むく 来る 人 ． ． ． 誰 、 この 頃 会う こと ある 、 想う 出せる ． ．...
11,rakugo,ご 隠居 \t おーい 、 定 、 定吉 定吉 \t ーーーー ぃ 、 ご 隠居 さん ...
12,rakugo,好き嫌い もの どなた ある もの 、 あれ 好き 、 これ 嫌い の これ また 一人ひと...


In [18]:
df.to_csv(csv_out, index=False)